# Construction of pYPK0_A_Athmod1

This notebook describe the construction of the _E. coli_ vector [pYPK0_A_Athmod1](pYPK0_A_Athmod1.gb).

![pYPK0_A plasmid](pYPK0_A.png "pYPK0_A plasmid")

A part of the [pydna](https://pypi.python.org/pypi/pydna/) package is imported in the code cell below.

In [1]:
from pydna.readers import read
from pydna.genbank import Genbank
from pydna.parsers import parse_primers
from pydna.amplify import pcr
from pydna.assembly import Assembly
from pydna.amplify import Anneal

The vector backbone [pYPK0](pYPK0.gb) is read from a local file.

In [2]:
pYPK0 = read("pYPK0.gb")

The restriction enzyme [AscI](http://rebase.neb.com/rebase/enz/AscI.html) is imported from [Biopython](http://biopython.org)

In [3]:
from Bio.Restriction import AscI

The plasmid is linearized with the enzyme.

In [4]:
pYPK0_AscI  = pYPK0.linearize(AscI)

Access to [Genbank](http://www.ncbi.nlm.nih.gov/nuccore) is needed in order to download the template.
If the email address below is not yours, change it before executing this script as you must always give
NCBI a way to contact you when using their service.

In [5]:
gb = Genbank("bjornjobb@gmail.com")

The template is downloaded from Genbank below.

In [6]:
template  = gb.nucleotide("NM_126612 REGION: 198..1370")
template

Gbank(NM_126612  198-1370)(-1173)

The two primers below are used to amplify the insert.

In [7]:
fp,rp =  parse_primers(""">1180_AthMod1_YPK_fw
                          GCCAGGTTGCCCACTTTCTCACTAGTGACCTGCAGCCGGAAATGGCAATGTCTGAATCCA
                          >1179_AthMod1_YPK_rv
                          TAAATCCGGATATCCTGATGCGTTTGTCTGCACAGATGGCTAATTCTTGCTGTTAAGGTC""")

The gene is amplifed using the primers specified above.

In [8]:
ins = pcr(fp, rp, template)

The primers anneal on the template like this.

In [9]:
ins.figure()

                                           5GGCAATGTCTGAATCCA...GACCTTAACAGCAAGAATTAG3
                                                                |||||||||||||||||||||
                                                               3CTGGAATTGTCGTTCTTAATCGGTAGACACGTCTGTTTGCGTAGTCCTATAGGCCTAAAT5
5GCCAGGTTGCCCACTTTCTCACTAGTGACCTGCAGCCGGAAATGGCAATGTCTGAATCCA3
                                            |||||||||||||||||
                                           3CCGTTACAGACTTAGGT...CTGGAATTGTCGTTCTTAATC5

A suggested PCR program.

In [10]:
ins.program()

|95°C|95°C               |    |tmf:56.0
|____|_____          72°C|72°C|tmr:56.2
|3min|30s  \ 56.6°C _____|____|45s/kb
|    |      \______/ 0:46|5min|GC 46%
|    |       30s         |    |1034bp

The linearzed vector and the insert are joined by homologous recombination.

In [11]:
asm = Assembly((pYPK0_AscI,ins))
asm

Assembly
fragments..: 5766bp 1034bp
limit(bp)..: 25
G.nodes....: 4
algorithm..: common_sub_strings

Usually two equally sized products are formed.

In [12]:
circular_products = asm.assemble_circular()
circular_products

[Contig(o6714), Contig(o6714)]

The first sequence is chosen.

In [13]:
candidate = circular_products[0]

The final vector is:

In [14]:
pYPK0_A_Athmod1 = candidate.synced("gttctgatcctcgagcatcttaagaattc")

A combination of standard primers and the gene specific primers are
used for the strategy to identify correct clones.
Standard primers are listed [here](standard_primers.txt).
The standard primers are read into a dictonary in the code cell below.

In [15]:
p = { x.id: x for x in parse_primers("standard_primers.txt") }

## Diagnostic PCR confirmation of pYPK0_A_Athmod1

The correct structure of pYPK0_A_Athmod1 is confirmed by PCR using standard primers
577 and 342 that are vector specific together with the Athmod1fw primer specific for the insert
in a multiplex PCR reaction with three primers present in the PCR reaction.

Two PCR products are expected if the insert was sucessfully cloned, sizes depending
on the orientation of the insert.
If the vector is empty, only one short product is formed.

## Expected PCR products sizes:

pYPK0_A_Athmod1 with insert in correct orientation.

In [16]:
Anneal( (p['577'], p['342'], fp), pYPK0_A_Athmod1).products

[Amplicon(1886), Amplicon(1708)]

Empty clone

In [17]:
Anneal( (p['577'], p['342'], fp), pYPK0).products

[Amplicon(934)]

The cseguid checksum for the resulting plasmid is calculated for future reference.
The [cseguid checksum](http://pydna.readthedocs.org/en/latest/pydna.html#pydna.utils.cseguid)
uniquely identifies a circular double stranded sequence.

In [18]:
pYPK0_A_Athmod1.cseguid()

_a1He4U5oWBuSCjZ4n9K0IUA5bs

The file is given a name based on the cloned insert

In [19]:
pYPK0_A_Athmod1.locus = "pYPK0_A_Athmod1"[:16]

Sequence is stamped with the cseguid checksum.
This can be used to verify the integrity of the sequence file.

In [20]:
pYPK0_A_Athmod1.stamp()

cSEGUID__a1He4U5oWBuSCjZ4n9K0IUA5bs

The sequence is written to a local file.

In [21]:
pYPK0_A_Athmod1.write("pYPK0_A_Athmod1.gb")

pYPK0_A_Athmod1.gb